In [1]:
#filename = "ml.py"
#exec(compile(open(filename, "rb").read(), filename, 'exec'))

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import dateutil.parser as parser
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from sklearn.cluster import KMeans

import tensorflow as tf

nltk.download('vader_lexicon')

# read the dataset into pandas dataframe
# df = pd.read_csv('./../downsampled_data', delim_whitespace=False).dropna()
df = pd.read_csv('./../downsampled_data.csv').dropna()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/trevorcarpenter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
df['categoryId'] = df['categoryId'].astype(np.float)

categories = pd.DataFrame(np.zeros((len(df), len(df['categoryId'].unique()))), columns = df['categoryId'].unique())
index = 0
for id in df['categoryId']:
    categories[id][index] = 1
    index += 1

In [5]:
# display the first five rows
retrieval_time = df['time_retrieved']
publish_time = df['publishedAt']
channel_publish_time = df['Channel_publishedAt']
retrieval_time_11_19_14 = df['11_19_14_update_timestamp']
columns_to_drop = ['Unnamed: 0', 'definition', 'categoryId', 'publishedAt', 'time_retrieved', 'Channel_title', '11_19_14_update_timestamp', 'Channel_publishedAt', 'video_id', 'channelId', 'thumbnail_link', 'Channel_country']
df = df.drop(columns_to_drop, axis = 1)

In [6]:
categories.columns

Float64Index([24.0, 25.0, 26.0, 17.0, 22.0, 20.0,  1.0, 10.0, 23.0, 28.0,  2.0,
              27.0, 19.0, 15.0, 29.0, 30.0],
             dtype='float64')

In [7]:
index = 0
for column in categories.columns:
    df.insert(df.shape[1], "Category_"+str(column), categories[column])
    index += 1

In [8]:
## time updates
import dateutil.parser as parser
age = []
age_update = []
channel_age = []
for i in df.index:
    channel_publish_time[i] = channel_publish_time[i].replace("\"", "")
    age.append(parser.isoparse(retrieval_time[i]) - parser.isoparse(publish_time[i]))
    age_update.append(parser.isoparse(retrieval_time_11_19_14[i]) - parser.isoparse(publish_time[i]))
    channel_age.append(parser.isoparse(channel_publish_time[i]) - parser.isoparse(publish_time[i]))
    if df['ratings_disabled'][i] == 'True':
        df['ratings_disabled'][i] = True
    elif df['ratings_disabled'][i] == 'False':
        df['ratings_disabled'][i] = False
        
    if df['Channel_hiddenSubscriberCount'][i] == 'True':
        df['Channel_hiddenSubscriberCount'][i] = True
    elif df['Channel_hiddenSubscriberCount'][i] == 'False':
        df['Channel_hiddenSubscriberCount'][i] = False
        
        
    if df['trended_later'][i] == 'True':
        df['trended_later'][i] = True
    elif df['trended_later'][i] == 'False':
        df['trended_later'][i] = False

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Applications/anaconda2/envs/newEn

In [9]:
## sentiment values

titles = df['title']
channel_title = df['channelTitle']
description = df['description']
channel_description = df['Channel_description']

title_sentiment_vals = []
channel_title_sentiment_vals = []
description_sentiment_vals = []
channel_description_sentiment_vals = []

sid = SentimentIntensityAnalyzer()
for sentence in titles:
    ss = sid.polarity_scores(str(sentence))
    title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_title:
    ss = sid.polarity_scores(str(sentence))
    channel_title_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in description:
    ss = sid.polarity_scores(str(sentence))
    description_sentiment_vals.append(ss['pos']-ss['neg'])
    
for sentence in channel_description:
    ss = sid.polarity_scores(str(sentence))
    channel_description_sentiment_vals.append(ss['pos']-ss['neg'])

In [10]:
df

,title,channelTitle,duration,trended_later,tags,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,...,Category_1.0,Category_10.0,Category_23.0,Category_28.0,Category_2.0,Category_27.0,Category_19.0,Category_15.0,Category_29.0,Category_30.0
0,Biden Victory Cold Open - SNL,Saturday Night Live,-0.0707230866960673,False,36.0,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.819940,0.504608,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,How President Trump and the White House reacte...,CNBC Television,-0.0856798827419832,False,28.0,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.274970,-0.350255,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Watch Live: President-Elect Joe Biden Addresse...,ABC News,0.09488158288091483,False,13.0,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.650191,0.461214,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,Alexa Rivera,-0.07104706061763949,False,0.0,False,"This was definitely such a fun prank, their re...",-0.258217,0.002973,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,beIN SPORTS USA,-0.07288291283988188,False,12.0,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330711,-0.407708,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875,US Election: Recount likely as candidates figh...,Sky News,-0.08497793924524347,False,19.0,False,The fight for the White House has intensified ...,-0.192403,-0.192300,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
876,The Packers should trade for Dallas&#39; WR Am...,The Herd with Colin Cowherd,-0.07536671290526863,False,33.0,False,Trade rumors are swirling around Dallas Cowboy...,-0.269840,-0.361884,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
877,Make You Mine This Season (Happiest Season),Tegan and Sara - Topic,-0.08692178277467658,False,1.0,False,Provided to YouTube by Facet/Warner Records Ma...,-0.340590,-0.440462,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
878,映画『ドクター・デスの遺産―BLACK FILE―』本編特別映像 2020年11月13日（金）公開,ワーナー ブラザース 公式チャンネル,-0.08395202182693154,False,35.0,False,綾野剛 × 北川景子 共演！ 警視庁捜査一課Ｎo.１コンビが挑む、 安楽死を手口にする連続殺...,-0.260114,-0.411267,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
from sklearn import preprocessing

df['title'] = title_sentiment_vals
df['channelTitle'] = channel_title_sentiment_vals
df['description'] = description_sentiment_vals
df['Channel_description'] = channel_description_sentiment_vals

le = preprocessing.LabelEncoder()
df['trending?'] = le.fit_transform(df['trending?'])
df['ratings_disabled'] = le.fit_transform(df['ratings_disabled'])
df['Channel_hiddenSubscriberCount'] = le.fit_transform(df['Channel_hiddenSubscriberCount'])
pd.set_option('display.max_columns', None)
df = df.dropna()

In [12]:
original_df = df.copy()

In [27]:
df = original_df.copy()

In [28]:
# dummy values for now - get these interactively later...
num_hidden_layers = 4
num_hidden_layer_nodes = [20, 20, 30, 20]
train_ratio = .7
hidden_layer_activations = ['sigmoid', 'sigmoid', 'sigmoid', 'sigmoid', 'sigmoid']
optimizer = 'adam'
learning_rate = .005
loss = 'mean_squared_error'
metrics = [tf.keras.metrics.Accuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.Precision()]
metrics_names = ["accuracy","recall","precision"]
epochs = 300
batch_size = 200
num_clusters = 3
original_trend_values = True

In [29]:
# Split the data into training and testing set by 70:30

In [32]:
# separate data into x and y - just random y for now..
y_column = 'trending?'
if not original_trend_values:
    df = df.drop(['trending?','trended_later'], axis = 1)
    clusters = KMeans(n_clusters=num_clusters).fit(df).labels_
    if 'cluster' in df.columns:
        df["cluster"]= clusters
    else:
        df.insert(df.shape[1], "cluster", clusters)
    y_column = 'cluster'
    
    
ratio = 0.7
train, test = train_test_split(df, train_size=ratio, random_state=42)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)

train_X = train.loc[:,train.columns != y_column]
train_Y = train[y_column]
test_X = test.loc[:,test.columns != y_column]
test_Y = test[y_column]



new_train_Y = train_Y
new_test_Y = test_Y

if not original_trend_values:
    new_train_Y = []
    new_test_Y = []
    potential_y_values = np.zeros((num_clusters, num_clusters))
    for c in range(num_clusters):
        potential_y_values[c][c] = 1
    for index in range(len(train_Y)):
        new_train_Y.append(potential_y_values[train_Y[index]])
    for index in range(len(test_Y)):
        new_test_Y.append(potential_y_values[test_Y[index]])
        

In [33]:
# build the logistic regression model - need clean data...
from sklearn.linear_model import LogisticRegression
LR_model = LogisticRegression(multi_class='ovr')
LR_model.fit(train_X, train_Y)

/Applications/anaconda2/envs/newEnv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(multi_class='ovr')

In [34]:
# confusion matrix
from sklearn.metrics import confusion_matrix

pred_Y = LR_model.predict(test_X);
confusion_matrix(test_Y, pred_Y)

array([[140,   4],
       [ 18,  48]])

In [21]:
# evaluate the accuracy of the LR model
accuracy = LR_model.score(test_X, test_Y)
print(accuracy)

0.9952380952380953


In [22]:
# build the ANN model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input

ANN_model = keras.Sequential()

# add hidden layers
for i in range(num_hidden_layers):
    if i == 0:
        ANN_model.add(Dense(num_hidden_layer_nodes[i], input_dim = train_X.shape[1], activation=hidden_layer_activations[i]))
    ANN_model.add(Dense(num_hidden_layer_nodes[i], activation=hidden_layer_activations[i]))

# add output layers
ANN_model.add(Dense(num_clusters, activation=hidden_layer_activations[num_hidden_layers]))

ANN_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
ANN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                720       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 30)                630       
_________________________________________________________________
dense_4 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 63        
Total params: 2,873
Trainable params: 2,873
Non-trainable params: 0
______________________________________________________

In [23]:
# train the model
ANN_model.fit(train_X, np.array(new_train_Y), validation_data=(test_X, np.array(new_test_Y)), epochs=epochs, batch_size=batch_size)

Train on 488 samples, validate on 210 samples
Epoch 1/300
488/488 [==============================] - 1s 2ms/sample - loss: 0.2548 - accuracy: 0.0000e+00 - recall: 0.7357 - precision: 0.3300 - val_loss: 0.2506 - val_accuracy: 0.0000e+00 - val_recall: 0.2381 - val_precision: 0.2381
Epoch 2/300
488/488 [==============================] - 0s 57us/sample - loss: 0.2482 - accuracy: 0.0000e+00 - recall: 0.2807 - precision: 0.2807 - val_loss: 0.2443 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/300
488/488 [==============================] - 0s 86us/sample - loss: 0.2424 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2390 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/300
488/488 [==============================] - 0s 84us/sample - loss: 0.2375 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2345 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00

Epoch 33/300
488/488 [==============================] - 0s 93us/sample - loss: 0.2203 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2175 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 34/300
488/488 [==============================] - 0s 77us/sample - loss: 0.2203 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2175 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 35/300
488/488 [==============================] - 0s 78us/sample - loss: 0.2202 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2175 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 36/300
488/488 [==============================] - 0s 89us/sample - loss: 0.2202 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2175 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision:

488/488 [==============================] - 0s 71us/sample - loss: 0.2151 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2127 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 66/300
488/488 [==============================] - 0s 79us/sample - loss: 0.2144 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2121 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 67/300
488/488 [==============================] - 0s 68us/sample - loss: 0.2136 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2113 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 68/300
488/488 [==============================] - 0s 67us/sample - loss: 0.2128 - accuracy: 0.0000e+00 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.2105 - val_accuracy: 0.0000e+00 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
E

Epoch 99/300
488/488 [==============================] - 0s 83us/sample - loss: 0.1467 - accuracy: 0.0000e+00 - recall: 0.3320 - precision: 0.9419 - val_loss: 0.1516 - val_accuracy: 0.0000e+00 - val_recall: 0.3143 - val_precision: 0.8919
Epoch 100/300
488/488 [==============================] - 0s 55us/sample - loss: 0.1450 - accuracy: 0.0000e+00 - recall: 0.3320 - precision: 0.9586 - val_loss: 0.1496 - val_accuracy: 0.0000e+00 - val_recall: 0.3095 - val_precision: 0.9028
Epoch 101/300
488/488 [==============================] - 0s 59us/sample - loss: 0.1432 - accuracy: 0.0000e+00 - recall: 0.3320 - precision: 0.9586 - val_loss: 0.1478 - val_accuracy: 0.0000e+00 - val_recall: 0.3143 - val_precision: 0.9041
Epoch 102/300
488/488 [==============================] - 0s 59us/sample - loss: 0.1415 - accuracy: 0.0000e+00 - recall: 0.3320 - precision: 0.9701 - val_loss: 0.1460 - val_accuracy: 0.0000e+00 - val_recall: 0.3143 - val_precision: 0.9429
Epoch 103/300
488/488 [==========================

488/488 [==============================] - 0s 71us/sample - loss: 0.1166 - accuracy: 0.0000e+00 - recall: 0.6598 - precision: 0.6970 - val_loss: 0.1223 - val_accuracy: 0.0000e+00 - val_recall: 0.6952 - val_precision: 0.7337
Epoch 134/300
488/488 [==============================] - 0s 70us/sample - loss: 0.1163 - accuracy: 0.0000e+00 - recall: 0.6578 - precision: 0.6963 - val_loss: 0.1222 - val_accuracy: 0.0000e+00 - val_recall: 0.6857 - val_precision: 0.7310
Epoch 135/300
488/488 [==============================] - 0s 67us/sample - loss: 0.1160 - accuracy: 0.0000e+00 - recall: 0.6598 - precision: 0.6970 - val_loss: 0.1220 - val_accuracy: 0.0000e+00 - val_recall: 0.6952 - val_precision: 0.7337
Epoch 136/300
488/488 [==============================] - 0s 68us/sample - loss: 0.1156 - accuracy: 0.0000e+00 - recall: 0.6598 - precision: 0.7000 - val_loss: 0.1217 - val_accuracy: 0.0000e+00 - val_recall: 0.6905 - val_precision: 0.7323
Epoch 137/300
488/488 [==============================] - 0s 52

488/488 [==============================] - 0s 60us/sample - loss: 0.1089 - accuracy: 0.0000e+00 - recall: 0.6926 - precision: 0.7116 - val_loss: 0.1159 - val_accuracy: 0.0000e+00 - val_recall: 0.7095 - val_precision: 0.7413
Epoch 168/300
488/488 [==============================] - 0s 70us/sample - loss: 0.1087 - accuracy: 0.0000e+00 - recall: 0.6926 - precision: 0.7116 - val_loss: 0.1158 - val_accuracy: 0.0000e+00 - val_recall: 0.7095 - val_precision: 0.7340
Epoch 169/300
488/488 [==============================] - 0s 72us/sample - loss: 0.1085 - accuracy: 0.0000e+00 - recall: 0.6926 - precision: 0.7116 - val_loss: 0.1157 - val_accuracy: 0.0000e+00 - val_recall: 0.7095 - val_precision: 0.7340
Epoch 170/300
488/488 [==============================] - 0s 53us/sample - loss: 0.1083 - accuracy: 0.0000e+00 - recall: 0.6906 - precision: 0.7110 - val_loss: 0.1156 - val_accuracy: 0.0000e+00 - val_recall: 0.7048 - val_precision: 0.7327
Epoch 171/300
488/488 [==============================] - 0s 53

488/488 [==============================] - 0s 57us/sample - loss: 0.0994 - accuracy: 0.0000e+00 - recall: 0.9611 - precision: 0.7651 - val_loss: 0.1072 - val_accuracy: 0.0000e+00 - val_recall: 0.9381 - val_precision: 0.7787
Epoch 202/300
488/488 [==============================] - 0s 53us/sample - loss: 0.0990 - accuracy: 0.0000e+00 - recall: 0.9631 - precision: 0.7655 - val_loss: 0.1065 - val_accuracy: 0.0000e+00 - val_recall: 0.9286 - val_precision: 0.7863
Epoch 203/300
488/488 [==============================] - 0s 66us/sample - loss: 0.0984 - accuracy: 0.0000e+00 - recall: 0.9447 - precision: 0.7696 - val_loss: 0.1059 - val_accuracy: 0.0000e+00 - val_recall: 0.9095 - val_precision: 0.7796
Epoch 204/300
488/488 [==============================] - 0s 74us/sample - loss: 0.0979 - accuracy: 0.0000e+00 - recall: 0.9201 - precision: 0.7649 - val_loss: 0.1054 - val_accuracy: 0.0000e+00 - val_recall: 0.9095 - val_precision: 0.7796
Epoch 205/300
488/488 [==============================] - 0s 56

488/488 [==============================] - 0s 64us/sample - loss: 0.0640 - accuracy: 0.0000e+00 - recall: 0.9816 - precision: 0.9657 - val_loss: 0.0716 - val_accuracy: 0.0000e+00 - val_recall: 0.9381 - val_precision: 0.9381
Epoch 236/300
488/488 [==============================] - 0s 55us/sample - loss: 0.0625 - accuracy: 0.0000e+00 - recall: 0.9775 - precision: 0.9656 - val_loss: 0.0705 - val_accuracy: 0.0000e+00 - val_recall: 0.9429 - val_precision: 0.9296
Epoch 237/300
488/488 [==============================] - 0s 55us/sample - loss: 0.0611 - accuracy: 0.0000e+00 - recall: 0.9775 - precision: 0.9578 - val_loss: 0.0694 - val_accuracy: 0.0000e+00 - val_recall: 0.9429 - val_precision: 0.9296
Epoch 238/300
488/488 [==============================] - 0s 68us/sample - loss: 0.0597 - accuracy: 0.0000e+00 - recall: 0.9775 - precision: 0.9578 - val_loss: 0.0680 - val_accuracy: 0.0000e+00 - val_recall: 0.9429 - val_precision: 0.9340
Epoch 239/300
488/488 [==============================] - 0s 69

488/488 [==============================] - 0s 58us/sample - loss: 0.0293 - accuracy: 0.0000e+00 - recall: 0.9959 - precision: 0.9939 - val_loss: 0.0426 - val_accuracy: 0.0000e+00 - val_recall: 0.9619 - val_precision: 0.9619
Epoch 270/300
488/488 [==============================] - 0s 78us/sample - loss: 0.0287 - accuracy: 0.0000e+00 - recall: 0.9918 - precision: 0.9918 - val_loss: 0.0423 - val_accuracy: 0.0000e+00 - val_recall: 0.9571 - val_precision: 0.9617
Epoch 271/300
488/488 [==============================] - 0s 60us/sample - loss: 0.0284 - accuracy: 0.0000e+00 - recall: 0.9898 - precision: 0.9918 - val_loss: 0.0424 - val_accuracy: 0.0000e+00 - val_recall: 0.9524 - val_precision: 0.9524
Epoch 272/300
488/488 [==============================] - 0s 65us/sample - loss: 0.0275 - accuracy: 0.0000e+00 - recall: 0.9918 - precision: 0.9918 - val_loss: 0.0423 - val_accuracy: 0.0000e+00 - val_recall: 0.9381 - val_precision: 0.9381
Epoch 273/300
488/488 [==============================] - 0s 54

In [24]:
# confusion matrix
pred_Y = ANN_model.predict_classes(test_X);
confusion_matrix(test_Y, pred_Y)

array([[81,  2,  5],
       [ 5, 44,  1],
       [ 0,  0, 72]])

In [26]:
# report evaluation metrics 
evaluated_metrics = ANN_model.evaluate(test_X, np.array(new_test_Y))
for i in range(len(metrics)):
    print(metrics_names[i] + ": %.2f" % evaluated_metrics[i])

210/210 [==============================] - 0s 215us/sample - loss: 0.0341 - accuracy: 0.0000e+00 - recall: 0.9381 - precision: 0.9381
accuracy: 0.03
recall: 0.00
precision: 0.94
